In [4]:
import pandas as pd
import os
from binance.client import Client
from backtesting.lib import crossover
import numpy as np

# stake_amount = amount of stake the bot will use for each trade 
# if stake_amount = "unlimited", this configuration will allow increasing/decreasing stakes depending on the performance
# of the bot (lower stake if the bot is losing, higher stakes if the bot has a winning record since higher balances are available), 
# and will result in profit compounding.
stake_amount_type = "unlimited"
# stake_amount_type = 500

# tradable percentage of the balance
# for example: if you want to run 3 bot instances (1h, 4h and 1D), you can set the percentage of the total balance to be allocated to each of the bots.
tradable_balance_ratio = 1 # 33%

# max number of open trades
# if tradable balance = 1000 and max_open_positions = 10, the stake_amount = 1000/10 = 100 
max_open_positions = 10

gTimeFrameNum = int("1")
gtimeframeTypeShort = "h"

api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

# Binance Client
client = Client(api_key, api_secret)

dfBestEMA = pd.read_csv('coinpairBestEma.csv')


def get_num_open_positions():
    try:
        # df_open_positions_1h = pd.read_csv('positions'+str(gTimeFrameNum)+gtimeframeTypeShort+'.csv')
        df_open_positions_1h = pd.read_csv('positions1h.csv')
        df_open_positions_1h = df_open_positions_1h[df_open_positions_1h.position == 1].Currency

        df_open_positions_4h = pd.read_csv('positions4h.csv')
        df_open_positions_4h = df_open_positions_4h[df_open_positions_4h.position == 1].Currency

        df_open_positions_1d = pd.read_csv('positions1d.csv')
        df_open_positions_1d = df_open_positions_1d[df_open_positions_1d.position == 1].Currency

        total_open_positions = len(df_open_positions_1h) + len(df_open_positions_4h) + len(df_open_positions_1d)
        return total_open_positions

    except Exception as e:
        msg = "get_num_open_positions - There was an error: "
        print(msg, e)
        # send_telegram_message(eWarning, msg+str(e))
        return -1

def get_open_positions(df):
    try:
        # df = pd.read_csv('positions'+str(gTimeFrameNum)+gtimeframeTypeShort+'2.csv')
        df_open_positions = df[df.position == 1]
        return df_open_positions

    except Exception as e:
        msg = "get_open_positions - There was an error: "
        print(msg, e)
        # sendTelegramMessage(eWarning, msg+e)
        return -1

def calc_stake_amount(coin):
    if stake_amount_type == "unlimited":
        num_open_positions = get_num_open_positions()

        if num_open_positions == -1:
            return 0
        if num_open_positions >= max_open_positions:
            return 0 

        balance = float(client.get_asset_balance(asset=coin)['free'])
        tradable_balance = balance*tradable_balance_ratio 
        stake_amount = tradable_balance/(max_open_positions-num_open_positions)
        return int(stake_amount)
    elif int(stake_amount_type) >= 0:
        return stake_amount_type
    else:
        return 0

def get_data(coinPair: str, aTimeframeNum: int, aTimeframeTypeShort: str, aFastMA=0, aSlowMA=0):

    try:
        # update EMAs from the best EMA return ratio
        global gFastMA
        global gSlowMA
        global gStrategyName

        lTimeFrame = str(aTimeframeNum)+aTimeframeTypeShort
        if aTimeframeTypeShort == "h":
            lTimeframeTypeLong = "hour"
        elif aTimeframeTypeShort == "d":
            lTimeframeTypeLong = "day"
        
        if aSlowMA > 0 and aFastMA > 0:
            gFastMA = aFastMA
            gSlowMA = aSlowMA
        else:
            listEMAvalues = dfBestEMA[(dfBestEMA.coinPair == coinPair) & (dfBestEMA.timeFrame == lTimeFrame)]

            if not listEMAvalues.empty:
                gFastMA = int(listEMAvalues.fastEMA.values[0])
                gSlowMA = int(listEMAvalues.slowEMA.values[0])
            else:
                gFastMA = int("0")
                gSlowMA = int("0")

        gStrategyName = str(gFastMA)+"/"+str(gSlowMA)+" EMA cross"

        # if bestEMA does not exist return empty dataframe in order to no use that trading pair
        if gFastMA == 0:
            frame = pd.DataFrame()
            return frame
        
        # if best Ema exist get price data 
        # lstartDate = str(1+gSlowMA*aTimeframeNum)+" "+lTimeframeTypeLong+" ago UTC"
        sma200 = 200+100
        lstartDate = str(sma200*aTimeframeNum)+" "+lTimeframeTypeLong+" ago UTC" 
        ltimeframe = str(aTimeframeNum)+aTimeframeTypeShort
        frame = pd.DataFrame(client.get_historical_klines(coinPair,
                                                        ltimeframe,
                                                        lstartDate))

        frame = frame[[0,4]]
        frame.columns = ['time','close']
        frame.close = frame.close.astype(float)
        frame.time = pd.to_datetime(frame.time, unit='ms')
        return frame
    except Exception as e:
        msg = "getdata - There was an error: "
        print(msg, e)
        # send_telegram_message(eWarning, msg+e)
        frame = pd.DataFrame()
        return frame 
        
def apply_technicals(df, aFastMA, aSlowMA):
    
    if aFastMA > 0: 
        df['fast_ema'] = df['close'].ewm(span=aFastMA, adjust=False).mean()
        df['slow_ema'] = df['close'].ewm(span=aSlowMA, adjust=False).mean()
        df['sma_50']   = df['close'].rolling(50).mean()
        df['sma_200']  = df['close'].rolling(200).mean()


coinPair = "ATOMBUSD"
gFastMA = int("8")
gSlowMA = int("34")
gStrategyName = str(gFastMA)+"/"+str(gSlowMA)+" CROSS"

df = get_data(coinPair, gTimeFrameNum, gtimeframeTypeShort)
df

,time,close
0,2023-01-14 23:00:00,12.880
1,2023-01-15 00:00:00,12.694
2,2023-01-15 01:00:00,12.585
3,2023-01-15 02:00:00,12.505
4,2023-01-15 03:00:00,12.574
...,...,...
295,2023-01-27 06:00:00,12.998
296,2023-01-27 07:00:00,13.022
297,2023-01-27 08:00:00,12.973
298,2023-01-27 09:00:00,12.984


In [5]:
num = get_num_open_positions()
num

28

In [63]:
print(gFastMA)
print(gSlowMA)
apply_technicals(df, gFastMA, gSlowMA)
df

45
65


,time,close,fast_ema,slow_ema,sma_50,sma_200
0,2023-01-12 02:00:00,11.836,11.836000,11.836000,NaN,NaN
1,2023-01-12 03:00:00,11.870,11.837478,11.837030,NaN,NaN
2,2023-01-12 04:00:00,11.953,11.842501,11.840545,NaN,NaN
3,2023-01-12 05:00:00,11.710,11.836740,11.836589,NaN,NaN
4,2023-01-12 06:00:00,11.776,11.834099,11.834753,NaN,NaN
...,...,...,...,...,...,...
295,2023-01-24 09:00:00,13.108,13.182404,13.130693,13.22244,12.635470
296,2023-01-24 10:00:00,13.117,13.179560,13.130278,13.22262,12.636790
297,2023-01-24 11:00:00,13.095,13.175884,13.129209,13.22406,12.638575
298,2023-01-24 12:00:00,13.092,13.172237,13.128082,13.22342,12.639920


In [51]:
df2 = df[(df.time >= "2023-01-20 18:00:00") & (df.time <= "2023-01-21 00:00:00")]
df2

,time,close,fast_ema,slow_ema,sma_50,sma_200
209,2023-01-20 18:00:00,0.9779,0.955636,0.963884,0.948374,0.967981
210,2023-01-20 19:00:00,0.9861,0.957979,0.964372,0.948744,0.968507
211,2023-01-20 20:00:00,1.0058,0.961658,0.965282,0.949516,0.969128
212,2023-01-20 21:00:00,1.0100,0.965376,0.966265,0.950416,0.969717
213,2023-01-20 22:00:00,1.0133,0.969063,0.967299,0.951612,0.970422
214,2023-01-20 23:00:00,1.0266,0.973489,0.968602,0.953130,0.971186
215,2023-01-21 00:00:00,1.0262,0.977544,0.969868,0.954588,0.971914


In [61]:
df2['order'] = ""
# for i in range(len(df2)):
for i in df2.index:
    # ignore first row
    if df2.index[0] == i:
        print("ignore first line")
        continue

    if df2.fast_ema[i-1] < df2.slow_ema[i-1] and df2.fast_ema[i] > df2.slow_ema[i]:
        df2.order[i] = "BUY"

    if df2.fast_ema[i-1] > df2.slow_ema[i-1] and df2.fast_ema[i] < df2.slow_ema[i]:
        df2.order[i] = "SELL"

df2


ignore first line


/tmp/ipykernel_221236/2881416697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['order'] = ""
/tmp/ipykernel_221236/2881416697.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.order[i] = "BUY"


,time,close,fast_ema,slow_ema,sma_50,sma_200,order
209,2023-01-20 18:00:00,0.9779,0.955636,0.963884,0.948374,0.967981,
210,2023-01-20 19:00:00,0.9861,0.957979,0.964372,0.948744,0.968507,
211,2023-01-20 20:00:00,1.0058,0.961658,0.965282,0.949516,0.969128,
212,2023-01-20 21:00:00,1.0100,0.965376,0.966265,0.950416,0.969717,
213,2023-01-20 22:00:00,1.0133,0.969063,0.967299,0.951612,0.970422,BUY
214,2023-01-20 23:00:00,1.0266,0.973489,0.968602,0.953130,0.971186,
215,2023-01-21 00:00:00,1.0262,0.977544,0.969868,0.954588,0.971914,


In [49]:
df2['order'] = ""
if crossover(df2.fast_ema, df2.slow_ema): 
    df2['order'] = "BUY"
elif crossover(df2.slow_ema, df2.fast_ema): 
    df2['order'] = "SELL"
df2


/tmp/ipykernel_221236/614666654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['order'] = ""


,time,close,fast_ema,slow_ema,sma_50,sma_200,order
209,2023-01-20 18:00:00,0.9779,0.955636,0.963884,0.948374,0.967981,
210,2023-01-20 19:00:00,0.9861,0.957979,0.964372,0.948744,0.968507,
211,2023-01-20 20:00:00,1.0058,0.961658,0.965282,0.949516,0.969128,
212,2023-01-20 21:00:00,1.0100,0.965376,0.966265,0.950416,0.969717,
213,2023-01-20 22:00:00,1.0133,0.969063,0.967299,0.951612,0.970422,
214,2023-01-20 23:00:00,1.0266,0.973489,0.968602,0.953130,0.971186,
215,2023-01-21 00:00:00,1.0262,0.977544,0.969868,0.954588,0.971914,
